# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import pandas as pd
import numpy as np

import pickle
import re
import nltk

from sqlalchemy import create_engine

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger', 'stopwords'])

[nltk_data] Downloading package punkt to /Users/ominguet/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ominguet/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ominguet/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ominguet/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('DisasterResponse', con=engine)

category_names = df.columns[4:]

X = df['message']
y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    """Normalize, tokenize and stem text string
    
    Args:
    text: string containing message for processing
       
    Returns:
    Tokens: list of strings. 
    List containing normalized and stemmed word tokens
    """

    urls = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls_find = re.findall(urls, text)
    for url in urls_find:
        text = text.replace(url, "urlplaceholder")

    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    tokens_cleaned = []
    for tok in tokens:
        token_s = [tok for tok in tokens if tok not in stopwords.words('english')]
        token_s = lemmatizer.lemmatize(tok).lower().strip()
        tokens_cleaned.append(token_s)

    return tokens_cleaned

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(class_weight='balanced')))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
pipeline.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
def multioutput_classification_report(y_true, y_pred):
    for i in range(0, len(category_names)):
        print(category_names[i])
        print("\t Accuracy: {:.4f}\t\t Precision: {:.4}\t\t Recall: {:.4}\t\t F1 Score:{:.4}".format(
            accuracy_score(y_true[:,i], y_pred[:,i]),
            precision_score(y_true[:,i], y_pred[:,i], average='weighted'),
            recall_score(y_true[:,i], y_pred[:,i], average='weighted'),
            f1_score(y_true[:,i], y_pred[:,i], average='weighted')
        ))

In [ ]:
y_pred_train = pipeline.predict(X_train)

In [ ]:
multioutput_classification_report(y_train, y_pred_train)

In [ ]:
y_pred_test = pipeline.predict(X_test)

In [ ]:
multioutput_classification_report(y_test, y_pred_test)

### 6. Improve your model
Use grid search to find better parameters. 

- https://scikit-learn.org/stable/modules/grid_search.html
- https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV

In [ ]:
parameters_rf = {
    'clf_estimator_estimators':[20,50]
}

cv_rf = GridSearchCV(estimator=pipeline, param_grid=parameters_rf, cv=5, scoring='f1_weighted', verbose=3)

In [ ]:
cv_rf.fit(X_train, y_train)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
y_pred_rf = cv_rf.predict(X_train)

In [ ]:
classification_report(y_train, y_pred_rf_train)

In [ ]:
y_pred_rf_test = cv_rf.predict(X_test)

In [ ]:
classification_report(y_test, y_pred_rf_test)

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

## AdaBoostClassifier
- https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostClassifier.html

In [ ]:
pipeline_ada = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])

In [ ]:
parameters_ada = {
    'clf_estimator_lr': [0.1, 0.3], #Learning Rate
    'clf_estimator_estimators': [100, 200]
}

In [ ]:
cv_ada = GridSearchCV(estimator=pipeline_ada, param_grid=parameters_ada, cv=5, scoring='f1_weighted', verbose=3)

In [ ]:
cv_ada.fit(X_train, y_train)

In [ ]:
y_pred_ada_train = cv_ada.predict(X_train)

In [ ]:
classification_report(y_train, y_pred_ada_train)

In [ ]:
y_pred_ada_test = cv_ada.predict(X_test)

In [ ]:
classification_report(y_test, y_pred_ada_test)

## SVM
- https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
pipeline_svm = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(SVClassifier()))
])

In [ ]:
parameters_svm = {
    'clf_estimator_kernel': ['sigmoid'], 
    'clf_estimator_gamma': ['scale'],
    'clf_estimator_c':[1,10,100]}

In [ ]:
cv_svm = GridSearchCV(estimator=pipeline_svm, param_grid=parameters_svm, cv=5, scoring='f1_weighted', verbose=3)

In [ ]:
cv_svm.fit(X_train, y_train)

In [ ]:
y_pred_svm_train = cv_svm.predict(X_train)

In [ ]:
classification_report(y_train, y_pred_svm_train)

In [ ]:
y_pred_svm_test = cv_svm.predict(X_test)

In [ ]:
classification_report(y_test, y_pred_svm_test)

### 9. Export your model as a pickle file

In [ ]:
with open('randomforest_cv.pkl', 'wb') as file:
    pickle.dump(cv_rf, file)

In [ ]:
with open('adaboost_cv.pkl', 'wb') as file:
    pickle.dump(cv_ada, file)

In [ ]:
with open('svm_cv.pkl', 'wb') as file:
    pickle.dump(cv_svm, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.